## Tauha 22i1239 - AI lab#8

# **Employee Shift Scheduling A Constraints Satisfiction problem**

You are tasked with developing an automated scheduling system that assigns employees to work shifts over a given period (e.g., one week). The assignment must satisfy several constraints to ensure fairness and operational efficiency.


### **Objectives**
- **CSP Modeling:** Formulate the scheduling problem as a CSP by defining variables, domains, and constraints.
- **Search Techniques:** Implement search strategies such as backtracking, along with heuristics like Minimum Remaining Value (MRV) and Most Constraining Variable (MCV).
- **Constraint Propagation:** Use techniques such as AC-3 to reduce the domain of possibilities for each shift before and during search.

### **Problem Description**:

**Schedule Structure**
- **Time Frame:** Consider a schedule spanning 5 days (Monday through Friday).
- **Shifts per Day:** Each day is divided into 3 shifts: Morning, Afternoon, and Night.
- **Employees:** You have a fixed list of employees available for scheduling.

### **Inputs**
1. **List of Employees:** An array of employee names (e.g., ["Alice", "Bob", "Carol", "Dave"]).
2. **Employee Availability:**: For each employee, a dictionary indicating the shifts or days they are available to work.

### **Constraints**
1. **Daily Work Limit:** Each employee can work at most one shift per day.

2. **Availability Constraint:** An employee can only be assigned to a shift if they are available for that day/shift.

### **Expected Output**
- **Schedule Grid:** A 5×3 grid (or equivalent structure) where each cell shows the assigned employee for that day and shift.
    For example:


    | Day       | Morning | Afternoon | Night |
    |-----------|---------|-----------|-------|
    | Monday    | Alice   | Carol     | Dave  |
    | Tuesday   | Carol   | Bob       | Dave  |
    | Wednesday | ...     | ...       | ...   |
    | Thursday  | ...     | ...       | ...   |
    | Friday    | ...     | ...       | ...   |

### **Feasibility:**
The produced schedule must satisfy all the constraints:
- No employee is scheduled more than once per day.
- Only available employees are assigned to shifts.


### **Overall Process**

1. **For Each Day in the Week:**
   - **Initialize Daily Variables and Domains:**
     - Build a `daySchedule` mapping each shift (e.g., Morning, Afternoon, Night) to an employee assignment (initially set to `None`).
     - Build `dayDomains` for each shift based on employee availability for that day.
   - **Run AC-3 for Constraint Propagation:**
     - Apply AC-3 to prune the domains of each shift using the "one shift per day" rule (i.e., no employee can work more than one shift per day).
   - **Perform Backtracking Search:**
     - **Select the Next Variable (Shift) using MRV:**  
       Choose the shift with the smallest remaining domain.
     - **Order Possible Assignments using LCV:**  
       Prioritize employee assignments that eliminate the fewest options for other shifts.
     - **Assign Employees and Check Consistency:**  
       Assign an employee to the chosen shift and verify that the assignment does not violate the constraints.
     - **Backtracking:**  
       If a dead-end occurs (i.e., a shift's domain becomes empty), backtrack to previous assignments and try alternative values.
   - **Store the Daily Schedule:**
     - Once all shifts for the day have been successfully assigned, store the day's schedule in the overall `weeklySchedule`.

2. **Return the Weekly Schedule:**
   - After processing every day, return the complete `weeklySchedule`.

In [45]:
# i221239 , AI lab 8 , cs-g
import random
######################################################

def shift_scheduler(days, shifts, employees, availability):
  weekly_schedule = {} #will return this

  for day in days:
    day_schedule = {}
    # Time = { employees , ... }
    day_domains = {shift: employees.copy() for shift in shifts}

    for employee in employees:
      for shift in shifts:
        # Only check if the employee is available on that day.
        if day in availability[employee]:
          # If the employee is available on that day but not for that specific shift, remove them from the domain.
          if shift not in availability[employee][day]:
            day_domains[shift].remove(employee)


    #this the part where bractracking is used
    is_scheduled = False
    taken = list()
    choosing_path = list()

    while not is_scheduled:
      #morning....
      if len(choosing_path) == 0:
        choice = random.randint(0,len(day_domains['Morning'])-1)
        choice_list = day_domains["Morning"]
        choosing_path.append(choice_list[choice])
        if choice_list[choice] not in taken:
          taken.append(choice_list[choice])
        else:
          choosing_path.pop()
      elif len(choosing_path) == 1:
        choice = random.randint(0,len(day_domains['Afternoon'])-1)
        choice_list = day_domains["Afternoon"]
        choosing_path.append(choice_list[choice])
        if (choice_list[choice]) not in taken:
          taken.append(choice_list[choice])
        else:
          choosing_path.pop()
      elif len(choosing_path) == 2:
        choice = random.randint(0,len(day_domains['Night'])-1)
        choice_list = day_domains["Night"]
        choosing_path.append(choice_list[choice])
        if (choice_list[choice]) not in taken:
          taken.append(choice_list[choice])
        else:
          choosing_path.pop()
      elif len(choosing_path) == 3:
        is_scheduled = True
        break
      else:
        is_scheduled = True
        break
      #print(choosing_path)
      #print(taken)

    weekly_schedule[day] = {}  # Create a dictionary for the current day

    # Assign employees to shifts for the current day
    weekly_schedule[day]['Morning'] = choosing_path[0]
    weekly_schedule[day]['Afternoon'] = choosing_path[1]
    weekly_schedule[day]['Night'] = choosing_path[2]

    #print(weekly_schedule)
    #print(choosing_path)
    #print(taken)

    #print(day , ' >> ' , day_domains)
    #print("---------------------------------")
    #print(weekly_schedule)


  return weekly_schedule

######################################################

def print_schedule(schedule):

  for day, shifts in schedule.items():
    print(f"{day}:")
    for shift, employee in shifts.items():
      print(f"  {shift}: {employee}")

  pass

#######################################################
# List of employees
employees = ["Alice", "Bob", "Carol", "Dave"]
# Employee availability dictionary
# Each employee is mapped to the days they are available, and for each day, the list of shifts they can work.
availability = {
    "Alice": {
        "Monday": ["Morning", "Afternoon", "Night"],
        "Wednesday": ["Morning", "Afternoon", "Night"],
        "Friday": ["Morning", "Afternoon", "Night"]
    },
    "Bob": {
        "Tuesday": ["Morning", "Afternoon", "Night"],
        "Thursday": ["Morning", "Afternoon", "Night"]
    },
    "Carol": {
        "Monday": ["Morning", "Afternoon"],
        "Tuesday": ["Morning"],
        "Wednesday": ["Afternoon", "Night"],
        "Thursday": ["Night"],
        "Friday": ["Morning", "Afternoon", "Night"]
    },
    "Dave": {
        "Monday": ["Night"],
        "Tuesday": ["Afternoon", "Night"],
        "Wednesday": ["Morning", "Afternoon"],
        "Thursday": ["Morning", "Afternoon", "Night"],
        "Friday": ["Afternoon"]
    }
}

# Define the days and shifts for the schedule (5 days, 3 shifts per day)
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
shifts = ["Morning", "Afternoon", "Night"]

#print(availability['Bob'][days[1]])

final_schedule = shift_scheduler(days,shifts,employees,availability)
print_schedule(final_schedule)


Monday:
  Morning: Bob
  Afternoon: Carol
  Night: Alice
Tuesday:
  Morning: Carol
  Afternoon: Dave
  Night: Bob
Wednesday:
  Morning: Dave
  Afternoon: Bob
  Night: Carol
Thursday:
  Morning: Alice
  Afternoon: Bob
  Night: Carol
Friday:
  Morning: Bob
  Afternoon: Carol
  Night: Alice
